# bilibili PCA UserCF

企图将番剧作为维度，进行主成分分析，之后降维，再进行聚类得到不同的人群

In [92]:
# import package
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd

In [93]:
AnimeDic = pd.read_json("D:/ML/animeId.json")[0].tolist()
def AnimeIdConvert(AnimeId):
    return AnimeDic.index(int(AnimeId))
#load rawdata
with open('D:/ML/alluserlocal.json', 'r',encoding="utf-8") as f:    
    # 逐行读取    
    data = (line.strip() for line in f)    
    # 格式化，然后每行作为list的一个元素    
    data_json = "[{0}]".format(','.join(data))
    recipes = pd.read_json(data_json)
raw_data = recipes["likevideo"].tolist()
tmp_data = []
for d in raw_data:
    if len(d) > 10:
        tmp_data.append(d)
raw_data = tmp_data
# change to shape(x, 3402)
data = []
for d in raw_data:
    arr = np.zeros(3402)
    for bungmi in d:
        try:
            arr[AnimeIdConvert(bungmi)] = 1
        except ValueError:
            continue
    data.append(arr)

In [94]:
# pca test
pca = PCA(n_components=30)
newd = pca.fit_transform(data)

使用聚类的效果非常的差，我们最终决定是比较新向量和每一个向量求取一个余弦相似度

In [128]:
my_favorite=[2661,645,5997,1547,2747,1293,5800,5798,1057,3048,5027,5793,6440,21557,963,427,177,3756]
convert_favorite = np.zeros(3402)
for ban in my_favorite:
    convert_favorite[AnimeIdConvert(ban)] = 1
convert_favorite = pca.transform([convert_favorite])

In [129]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
max_vec = newd[0]
max_similar = 0.0
for user in newd:
    similar = cosine_similarity([user],convert_favorite)
    if similar > max_similar:
        max_similar = similar
        max_vec = user
simi = pca.inverse_transform(max_vec)
#%matplotlib inline
#plt.hist(simi)
order = np.argsort(simi)
order = order[::-1]

In [133]:
def ToAnimeId(index):
    return AnimeDic[index]
count = 0
i = 0
while(count < 10):
    if ToAnimeId(order[i]) in my_favorite:
        i += 1
        continue
    else:
        print(ToAnimeId(order[i]))
        i += 1
        count += 1

6446
1699
1512
835
1539
6465
6001
425
5996
5776
